# 타이타닉 데이터 전처리

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score

import joblib

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## X-data 전처리
**Pclass**: [ 1, 2, 3 ]   
**Name**: [ 'Master.', 'Miss.', 'Mr.', 'Mrs.' ]   
**Sex**: [ 'male', 'female' ]   
**Age**: [ (-0.08, 16.0], (16.0, 32.0], (32.0, 48.0], (48.0, 64.0], (64.0, 80.0] ]   
**Fare**: [ (-0.001, 7.854], (7.854, 10.5], (10.5, 21.679], (21.679, 39.688], (39.688, 512.329] ]   
**Cabin**: [ 'T', 'A', 'B', 'C', 'D', 'E', 'F', 'G' ]   
**Embarked**: ['S', 'C', 'Q']   
**Family**: [0, 1]   
<br>
모든 범주형 데이터는 int형 변환 후 One-Hot Encoding 적용   
**PassngerId** 및 **Survived** 열은 X-data 사용 시 제거

In [3]:
data_df = df.copy()

data_df.replace(['male','female'], [0,1], inplace=True)

# 형제 또는 부모의 수보다 가족의 유무 그 자체가 의미 있다고 판단하여 범주형 데이터로 변환
data_df['Family'] = data_df['SibSp'] + data_df['Parch']
data_df['Family'] = data_df['Family'].apply(lambda x: 1 if x else 0)

# 운임은 경제 수준을 나타내는 상대적인 지표이기에 Quantile을 기준으로 5개 구간으로 분리
# 운임을 일정한 구간으로 구분할 시 특정 구간에 쏠림 현상이 발생하여 Qunatile을 기준으로 분리
data_df['Fare'] = pd.qcut(data_df['Fare'], 5, labels=list(range(5)))

# 탑승지의 결측치 2개가 밖에 없기 때문에 최빈값 S로 채움
data_df['Embarked'] = data_df['Embarked'].fillna('S')
data_df.replace(['S','C','Q'], [0,1,2], inplace=True)

# T 선실이 최상층, A부터 G 순서대로 낮은 층의 선실
data_df['Cabin'] = data_df['Cabin'].str[0]
data_df.replace(['T','A','B','C','D','E','F','G'], range(8), inplace=True)

# Value Counts를 보았을 때 의미있다고 판단되는 4개 데이터 Master. Miss. Mr. Mrs.를 범주로 지정
# Master.와 Miss.는 각각 18세 미만 남성/여성, Mr.와 Mrs.는 각각 18세 이상 남성/여성
data_df['Name'] = data_df['Name'].apply(lambda x: str(x).split(',')[1].split()[0])
data_df.replace(['Master.','Mr.','Miss.','Mrs.'], range(4), inplace=True)

# Mlle.은 마드모아젤(Miss.와 동일), Ms.는 Miss.의 약어, Mme.는 마담(Mrs.와 동일)
data_df['Name'] = data_df['Name'].replace(['Mlle.', 'Ms.'], 2)
data_df['Name'] = data_df['Name'].replace('Mme.', 3)

# 나머지 이름은 나이와 성별에 따라 Rule 적용
# 조건에 맞지 않는 이름을 가진 행 중 유일하게 나이가 결측치인 766번 행의 경우,
# Dr.라는 이름과 male이란 성별을 통해 Mr.라는 이름이 유추되기 때문에 그대로 진행
for i in range(len(data_df)):
    if data_df['Name'][i] not in range(4):
        data_df['Name'][i] = data_df['Sex'][i] + (0 if data_df['Age'][i] < 18 else 1)
data_df['Name'] = data_df['Name'].astype(int)

data_df = data_df.drop(['SibSp','Parch','Ticket'], axis=1)

data_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,Fare,Cabin,Embarked,Family
0,1,0,3,1,0,22.0,0,NaN,0,1
1,2,1,1,3,1,38.0,4,3.0,1,1
2,3,1,3,2,1,26.0,1,NaN,0,0
3,4,1,1,3,1,35.0,4,3.0,0,1
4,5,0,3,1,0,35.0,1,NaN,0,0


### 나이 예측 모델 생성 및 적용

#### 나이 결측치를 평균값이 아닌 예측 모델로 적용한 이유
1. 전체 행에 비해 결측치가 많아 해당 행을 제외하기가 어려움
2. 나이 결측치를 평균값으로 대체했을 때보다 예측 모델을 적용했을 때 더 높은 성능
> 나이 결측치를 평균값 처리한 후 실제 모델(로지스틱 회귀)에 넣어봤을 때 Accuracy: 0.8059701492537313   
> 나이 결측치를 예측한 후 실제 모델(로지스틱 회귀)에 넣어봤을 때 Accuracy: 0.8208955223880597

In [4]:
# Feature는 모든 조합에 대해 for문을 돌렸을 때,
# 로지스틱 회귀에서 Accuracy가 높고 Value Counts를 확인했을 때도 의미있다고 판단한 최선책을 선택
categorical_transformer = OneHotEncoder(categories='auto', handle_unknown='ignore')
categorical_features = ['Name', 'Sex', 'Fare']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)])

age_pipe = Pipeline(steps=[('preprocessor', preprocessor)])

In [5]:
age_df = data_df[ data_df['Age'] > 0 ].copy()

age_X = age_df.drop(['PassengerId','Survived','Pclass','Age',
                      'Cabin','Embarked','Family'], axis=1)
age_X = age_pipe.fit_transform(age_X)
age_Y = np.array(age_df[['Age']])

# 데이터의 수 자체가 적기 때문에 Train/Test Split을 진행하지 않음
age_model = LinearRegression()
age_model.fit(age_X, age_Y)

LinearRegression()

In [6]:
test_age_df = data_df[ data_df['Age'].isnull() ].copy()

test_age_X = age_pipe.transform(test_age_df)
pred_age_y = age_model.predict(test_age_X)

for i, pred in zip(test_age_df.index.tolist(), pred_age_y):
    data_df['Age'][i] = pred

data_df['Age'] = data_df['Age'].astype(int)

# 나이는 건강 상태 및 대피 순서를 나타내는 절대적인 지표이기에 전체 범위를 5개 구간으로 분리
# 운임과 다르게 특정 구간에 데이터가 쏠리지 않기 때문에 Qunatile을 기준으로 분리하지 않음
data_df['Age'] = pd.cut(data_df['Age'], 5, labels=list(range(5)))

data_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,Fare,Cabin,Embarked,Family
0,1,0,3,1,0,1,0,NaN,0,1
1,2,1,1,3,1,2,4,3.0,1,1
2,3,1,3,2,1,1,1,NaN,0,0
3,4,1,1,3,1,2,4,3.0,0,1
4,5,0,3,1,0,2,1,NaN,0,0


### 나이 예측 모델 결과 확인

In [7]:
test_age_df['Age'] = pred_age_y
test_age_df['Age'] = test_age_df['Age'].astype(int)

In [8]:
test_age_df[test_age_df['Name'] == 0]['Age'].value_counts() # 18세 미만 남성 그룹

0    2
8    1
5    1
Name: Age, dtype: int64

In [9]:
test_age_df[test_age_df['Name'] == 1]['Age'].value_counts() # 18세 이상 남성 그룹

30    77
36    18
31    14
38    11
Name: Age, dtype: int64

In [10]:
test_age_df[test_age_df['Name'] == 2]['Age'].value_counts() # 18세 미만 여성 그룹

18    20
24     8
26     4
19     4
Name: Age, dtype: int64

In [11]:
test_age_df[test_age_df['Name'] == 3]['Age'].value_counts() # 18세 이상 여성 그룹

31    8
39    8
37    1
Name: Age, dtype: int64

### 선실 예측 모델 생성 및 적용

#### 선실 결측치에 예측 모델을 적용한 이유
1. 선실 정보는 생존률과 매우 밀접한 타이타닉 내 위치 정보라 판단하여 최대한 살리는 방향으로 결정
2. 선실 열을 제거했을 때보다 예측 모델을 적용했을 때 더 높은 성능
> 선실 열을 제거한 후 실제 모델(로지스틱 회귀)에 넣어봤을 때 Accuracy: 0.8208955223880597   
> 선실 결측치를 예측한 후 실제 모델(로지스틱 회귀)에 넣어봤을 때 Accuracy: 0.8432835820895522

In [12]:
# Feature는 모든 조합에 대해 for문을 돌렸을 때,
# 로지스틱 회귀에서 Accuracy가 높고 Value Counts를 확인했을 때도 의미있다고 판단한 최선책을 선택
categorical_transformer = OneHotEncoder(categories='auto', handle_unknown='ignore')
categorical_features = ['Pclass', 'Age', 'Fare', 'Family']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)])

cabin_pipe = Pipeline(steps=[('preprocessor', preprocessor)])

In [13]:
cabin_df = data_df[ data_df['Cabin'].notnull() ].copy()

cabin_X = cabin_df.drop(['PassengerId','Survived','Name',
                      'Sex','Cabin','Embarked'], axis=1)
cabin_X = cabin_pipe.fit_transform(cabin_X)
cabin_Y = np.array(cabin_df[['Cabin']])

# 데이터의 수 자체가 매우 적기 때문에 Train/Test Split을 진행하지 않음
cabin_model = LinearRegression()
cabin_model.fit(cabin_X, cabin_Y)

LinearRegression()

In [14]:
test_cabin_df = data_df[ data_df['Cabin'].isnull() ].copy()

test_cabin_X = cabin_pipe.transform(test_cabin_df)
pred_cabin_y = cabin_model.predict(test_cabin_X)

for i, pred in zip(test_cabin_df.index.tolist(), pred_cabin_y):
    data_df['Cabin'][i] = pred

data_df['Cabin'] = data_df['Cabin'].astype(int)

data_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,Fare,Cabin,Embarked,Family
0,1,0,3,1,0,1,0,6,0,1
1,2,1,1,3,1,2,4,3,1,1
2,3,1,3,2,1,1,1,6,0,0
3,4,1,1,3,1,2,4,3,0,1
4,5,0,3,1,0,2,1,6,0,0


### 선실 예측 모델 결과 확인
선실(갑판)에 관한 정보는 [참고](https://ko.wikipedia.org/wiki/RMS_타이타닉#층별_구조)

In [15]:
test_cabin_df['Cabin'] = pred_cabin_y
test_cabin_df['Cabin'] = test_cabin_df['Cabin'].astype(int)

In [16]:
test_cabin_df[test_cabin_df['Pclass'] == 1]['Cabin'].value_counts()

3    24
2    14
1     2
Name: Cabin, dtype: int64

In [17]:
test_cabin_df[test_cabin_df['Pclass'] == 2]['Cabin'].value_counts()

4    70
5    50
6    48
Name: Cabin, dtype: int64

In [18]:
test_cabin_df[test_cabin_df['Pclass'] == 3]['Cabin'].value_counts()

6    259
5    191
7     26
4      3
Name: Cabin, dtype: int64

## 데이터 확인 및 저장

In [19]:
data_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,Fare,Cabin,Embarked,Family
0,1,0,3,1,0,1,0,6,0,1
1,2,1,1,3,1,2,4,3,1,1
2,3,1,3,2,1,1,1,6,0,0
3,4,1,1,3,1,2,4,3,0,1
4,5,0,3,1,0,2,1,6,0,0


In [20]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int64   
 1   Survived     891 non-null    int64   
 2   Pclass       891 non-null    int64   
 3   Name         891 non-null    int64   
 4   Sex          891 non-null    int64   
 5   Age          891 non-null    category
 6   Fare         891 non-null    category
 7   Cabin        891 non-null    int64   
 8   Embarked     891 non-null    int64   
 9   Family       891 non-null    int64   
dtypes: category(2), int64(8)
memory usage: 58.0 KB


In [21]:
data_df.set_index('PassengerId').to_csv(f'titanic_data.csv')

## Pipeline 생성 및 저장

In [22]:
categorical_transformer = OneHotEncoder(categories='auto', handle_unknown='ignore')
categorical_features = ['Pclass', 'Name', 'Sex', 'Age',
                        'Fare', 'Cabin', 'Embarked', 'Family']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)])

pipe = Pipeline(steps=[('preprocessor', preprocessor)])

In [23]:
joblib.dump(pipe, 'titanic_pipe.pkl', compress=True)

['titanic_pipe.pkl']